# AoC Day 14

Jenna Jordan

15 December 2021

## Prompt

--- Day 14: Extended Polymerization ---

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

```
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

- The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
- The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
- The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

```
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [1]:
# get input

with open("inputs/day14.txt") as file:
    input_polymer = file.readline().rstrip()
    file.readline()
    input_rules = file.read().splitlines()
    
test_input_polymer = "NNCB"
test_input_rules = ["CH -> B", "HH -> N", "CB -> H", "NH -> C", "HB -> C", "HC -> B", "HN -> C", "NN -> C", "BH -> H", "NC -> B", "NB -> B", "BN -> B", "BB -> N", "BC -> B", "CC -> N", "CN -> C"]

In [2]:
def get_inserts(input_rules):
    rules = {}
    for r in input_rules:
        r_split = r.split(" -> ")
        pair = r_split[0]
        insert = r_split[1]
        new_string = pair[0] + insert + pair[1]
        rules[pair] = new_string
    return rules

In [3]:
def get_polymer_pairs(polymer):
    pairs = []
    for i, letter in enumerate(polymer[:-1]):
        pair = polymer[i:i+2]
        pairs.append(pair)
    return pairs

In [4]:
def execute_one_insertion_step(polymer, rules):
    new_polymer = ""
    polymer_pairs = get_polymer_pairs(polymer)
    inserts = get_inserts(rules)
    for p in polymer_pairs:
        if p in inserts.keys():
            if new_polymer:
                new_polymer += inserts[p][1:]
            else:
                new_polymer += inserts[p]
    return new_polymer

In [5]:
def execute_x_insertion_steps(polymer, rules, steps):
    for s in range(steps):
        polymer = execute_one_insertion_step(polymer, rules)
    return polymer

In [6]:
from collections import Counter

In [7]:
def calculate_minmax_diff(polymer):
    count_elements = Counter(polymer)
    most_common = count_elements.most_common()
    diff = most_common[0][1] - most_common[-1][1]
    return diff

In [8]:
test_10step_polymer = execute_x_insertion_steps(test_input_polymer, test_input_rules, 10)
calculate_minmax_diff(test_10step_polymer)

1588

In [9]:
real_10step_polymer = execute_x_insertion_steps(input_polymer, input_rules, 10)
calculate_minmax_diff(real_10step_polymer)

3213

### Part 1 Solution

3213 - CORRECT!

--- Part Two ---

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [10]:
def get_new_pair_count(old_pair_count, pair_insert_rules):
    new_pair_count = {}
    for pair in old_pair_count.keys():
        p1, p2 = pair_insert_rules[pair]
        old_count = old_pair_count[pair]
        if p1 in new_pair_count.keys():
            new_pair_count[p1] += old_count
        else:
            new_pair_count[p1] = old_count
        if p2 in new_pair_count.keys():
            new_pair_count[p2] += old_count
        else:
            new_pair_count[p2] = old_count
    return new_pair_count

In [11]:
def get_pair_inserts(input_rules):
    rules = {}
    for r in input_rules:
        r_split = r.split(" -> ")
        pair = r_split[0]
        insert = r_split[1]
        new_pair1 = pair[0] + insert
        new_pair2 = insert + pair[1]
        rules[pair] = (new_pair1, new_pair2)
    return rules

In [12]:
def pairs_after_x_steps(polymer, rules, steps):
    initial_pairs = get_polymer_pairs(polymer)
    pair_insert_rules = get_pair_inserts(rules)
    pair_count = {}
    for pair in initial_pairs:
        if pair in pair_count.keys():
            pair_count[pair] += 1
        else:
            pair_count[pair] = 1
    for s in range(steps):
        pair_count = get_new_pair_count(pair_count, pair_insert_rules)
    return pair_count

In [13]:
def count_letters_in_pairs(pair_count):
    letter_count = {}
    for pair in pair_count.keys():
        l1 = pair[0]
        l2 = pair[1]
        if l1 in letter_count.keys():
            letter_count[l1] += pair_count[pair]
        else:
            letter_count[l1] = pair_count[pair]
        if l2 in letter_count.keys():
            letter_count[l2] += pair_count[pair]
        else:
            letter_count[l2] = pair_count[pair]
    
    return letter_count

In [14]:
def final_minmax_calc(letter_count, polymer):
    letter_count[polymer[0]] += 1
    letter_count[polymer[-1]] += 1
    
    for letter in letter_count.keys():
        letter_count[letter] //= 2
    
    return max(letter_count.values()) - min(letter_count.values())

In [15]:
test_pair_count = pairs_after_x_steps(test_input_polymer, test_input_rules, 40)

In [16]:
letter_count_test = count_letters_in_pairs(test_pair_count)

In [17]:
final_minmax_calc(letter_count_test, test_input_polymer)

2188189693529

In [18]:
pair_count = pairs_after_x_steps(input_polymer, input_rules, 40)

In [19]:
letter_count = count_letters_in_pairs(pair_count)

In [20]:
final_minmax_calc(letter_count, input_polymer)

3711743744429

### Part 2 Solution

3711743744429 - CORRECT!